In [ ]:
!pip install openai langchain streamlit faiss-cpu python-dotenv requests pandas

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o", temperature=0.2, openai_api_key=OPENAI_API_KEY)

In [ ]:
import textwrap

def fetch_web_stub(topic: str):

    sample = f"""
    {topic} is a current civic and policy challenge. It affects vulnerable groups, especially residents with low income,
    immigrants with language-access barriers, and people with disabilities. Governments are trying to improve service delivery
    but face constraints like funding, data quality, and outdated processes.
    """
    return [sample]

def chunk_text(docs, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    all_chunks = []
    for d in docs:
        all_chunks.extend(splitter.split_text(d))
    return all_chunks

In [ ]:
def summarize_chunk(chunk):
    prompt = f"""
    You are assisting a civic innovation team. Summarize the following text in 4-5 sentences.
    Keep it factual and civic-oriented.

    Text:
    {chunk}
    """
    resp = llm([HumanMessage(content=prompt)])
    return resp.content.strip()

def summarize_all(chunks):
    summaries = [summarize_chunk(c) for c in chunks]
    return "\n".join(summaries)

In [ ]:
def generate_policy_brief(topic, context):
    prompt = f"""
    You are an AI for Impact assistant helping a public innovation team at a university center.
    Based on the context below, create a structured, equity-aware policy brief.

    Topic: {topic}

    Context:
    {context}

    Your output MUST follow this JSON structure:

    {{
      "topic": "...",
      "summary": "...",
      "affected_populations": ["...", "..."],
      "stakeholders": ["government agencies", "NGOs", "community partners", "residents"],
      "policy_options": [
        {{
          "name": "Option 1",
          "description": "...",
          "pros": ["..."],
          "cons": ["..."],
          "equity_impact": "..."
        }},
        {{
          "name": "Option 2",
          "description": "...",
          "pros": ["..."],
          "cons": ["..."],
          "equity_impact": "..."
        }}
      ],
      "risks": ["..."],
      "data_gaps": ["..."],
      "recommended_next_steps": ["user testing with affected groups", "check legal/policy alignment"]
    }}

    Make sure options are realistic for a state agency, city department, or nonprofit partner.
    """
    resp = llm([HumanMessage(content=prompt)])
    return resp.content

In [ ]:
topic = "paratransit accessibility for disabled riders in Massachusetts"
docs = fetch_web_stub(topic)
chunks = chunk_text(docs)
context = summarize_all(chunks)
brief = generate_policy_brief(topic, context)
print(brief)

In [ ]:
import streamlit as st

st.title("PolicySynth Lite 2.0 – Equity AI")

topic = st.text_input("Enter a civic / policy topic", "language access for immigrant families in Massachusetts")

if st.button("Generate Brief"):
    docs = fetch_web_stub(topic)
    chunks = chunk_text(docs)
    context = summarize_all(chunks)
    brief = generate_policy_brief(topic, context)
    st.json(brief)